In [2]:
import random
import math
import numpy as np
import copy
import getopt
import psycopg2
import sys
import time
import matplotlib.pyplot as plt


In [3]:
def readInput(mode):
    global D
    global n
    #mode 0 bank, mode 1 adult
    if mode==0:
        input_path = "./bank_marketing.csv"
        input_file = open(input_path,'r')
        n = len(input_file.readlines())-1
        input_file.close()
        D = np.zeros(n)  
        i = 0
        input_file = open(input_path,'r')
        for line in input_file.readlines():
                elements = line.split(",")
                if i>0:
                    D[i-1] = int(elements[5])
                i+=1

        D.sort()
    if mode==1:
        input_path = "./adult.csv"
        input_file = open(input_path,'r')
        n = len(input_file.readlines())-1
        input_file.close()
        D = np.zeros(n)  
        i = 0
        input_file = open(input_path,'r')
        for line in input_file.readlines():
                elements = line.split(",")
                if i>0:
                    D[i-1] = int(elements[2])
                i+=1
        D.sort()


In [4]:
def discretize(D, b):    
    n = len(D)
    discreteD = np.zeros(n)
    for i in range(n):
        discreteD[i] = int(D[i]/b)
    return discreteD

def shiftD(D,x):
    #shift towards left
    return D-x
    
def count(D, a, b):
    counter = 0
    n = len(D)
    for i in range(n):
        if D[i] >=a and D[i]<=b:
            counter+=1
    return counter

def clip(D, a, b):
    clipped = copy.deepcopy(D)
    clipped[clipped<a] = a
    clipped[clipped>b] = b
    return clipped

def LapNoise():
    a = random.uniform(0,1)
    b = math.log(1/(1-a))
    c = random.uniform(0,1)
    if c>0.5:
        return b
    else:
        return -b
    
def finiteDomainMedian(eps, beta, a,b, D):
    #[a,b] denotes range
    n = len(D)
    u = np.zeros(n+2)#utility score
    l = np.zeros(n+3)#score changing point
    l[0] = a
    for i in range(n+2):
        if i<=n/2:
            u[i] = -n/2-1+i
            l[i+1]= D[i]
        u[int(n/2+1)] = 0
        l[int(n/2+2)] = D[int(n/2)]
        if i>n/2+1:
            u[i] = n/2+1-i
            l[i]= D[i-2]
            
    l[n+2] = b
    weight = np.zeros(n+2)
    for i in range(n+2):
        weight[i] = (l[i+1]-l[i])*math.pow(np.e, eps*u[i]/2)
    weight[int(n/2+1)] = math.pow(np.e, 0)
    
    totalWeight = sum(weight)
    weight = weight/totalWeight

    i = np.random.choice(list(range(n+2)), p=weight)
    if i==int(n/2+1):
        return l[i]
    else:
        x = np.random.randint(l[i], l[i+1])
        return x

In [5]:
def unboundedDomainRadius(eps, beta, D):
    TNoise = 2/eps*LapNoise()
    T = len(D)
    i=1
    while True:
        Ti = T-4/eps*np.log(2*i**2 * np.pi**2/(3*beta)) - 2/eps*np.log(4/beta)+TNoise
        if i==1:
            Qi = count(abs(D),0,0)+4/eps*LapNoise()
        else:
            Qi = count(abs(D),0, math.pow(2, i-2))+4/eps*LapNoise()

        if Qi>Ti:
            break
        else:
            i+=1
            
    if i==1:
        return 0
    else:
        return math.pow(2,i-1)
    
def unboundedDomainShiftedMean(eps, beta, b, D):
    n = len(D)
    intD = discretize(D,b)
    rad = unboundedDomainRadius(eps/4, beta/4, intD)
    clipped = clip(intD,-rad,rad)
    interiorPoint = finiteDomainMedian(eps, beta, -rad, rad, intD)
    shifted = clipped - interiorPoint
    width = unboundedDomainRadius(eps/4, beta/4, shifted)

    DforMean = clip(D, interiorPoint*b -width*b-2*b, interiorPoint*b +width*b+2*b)
    noisyMu = sum(DforMean)/n+8*(width*b+b)/(eps*n)*LapNoise()
    error = abs(noisyMu-sum(D)/n)
    #print("ratio is "+ str(8*(width*b+b)*np.log(4/beta)/(eps*n)/error))
    indicator = 0
    if sum(D)/n<noisyMu +8*(width*b+b)*np.log(4/beta)/(eps*n) and sum(D)/n>noisyMu - 8*(width*b+b)*np.log(4/beta)/(eps*n):
        indicator = 1
    #return [noisyMu - 8*(width*b+b)*np.log(4/beta)/(eps*n),noisyMu + 8*(width*b+b)*np.log(4/beta)/(eps*n) ]
    return noisyMu, 8*(width*b+b)*np.log(4/beta)/(eps*n), indicator

In [8]:
readInput(0)
b=1
eps=1
beta=0.1
truncate = int(0.05*n)
D = D[truncate:n-truncate]
n = len(D)
print(n)
width = D[n-1]-D[0]
intD = discretize(D,b) 
trueMean = sum(D)/n
print(width)
print(count(D, D[0], D[0]+width/4))
print(count(D, D[n-1]-width/4, D[n-1]))
print(trueMean)
D[n-1]

40691
5940.0
30836
1337
930.2625396279276


5768.0

In [12]:
errors = []
lengths = []
correct_count = 0
left = []
right = []
est = []
num_repeat=100
for i in range(num_repeat):
    noisyMu, length,indi = unboundedDomainShiftedMean(eps, beta, b, D)
    left.append(noisyMu-length)
    right.append(noisyMu+length)
    est.append(noisyMu)
    correct_count+=indi
    errors.append(abs(noisyMu-trueMean))
    lengths.append(length)
    
    
print("correct rate = " +str(correct_count/num_repeat))
avgLength = sum(lengths)/num_repeat
errors.sort()
errorQuantile = errors[int(num_repeat*0.9)]
print("Average CI length = "+str(avgLength))
print("90% error is "+ str(errorQuantile))
print("CI length/90% error = "+str(avgLength/errorQuantile))

correct rate = 0.96
Average CI length = 351.9794203772585
90% error is 266.7022495326528
CI length/90% error = 1.3197467250240238
